In [ ]:
import cdsapi

c = cdsapi.Client()
c.retrieve(
    "reanalysis-era5-complete", 
    {
        "date": "20130101",  # The hyphens can be omitted
        "levelist": "131/to/137",  # 1 is top level, 137 the lowest model level in ERA5. Use '/' to separate values.
        "levtype": "ml",
        "param": "131/132",  # Full information at https://apps.ecmwf.int/codes/grib/param-db/
        # The native representation for temperature is spherical harmonics
        "stream": "oper",  # Denotes ERA5. Ensemble members are selected by 'enda'
        "time": "00/to/23/by/1",
        "type": "an",
        "grid": "0.25/0.25",  # Latitude/longitude. Default: spherical harmonics or reduced Gaussian grid
        "format": "netcdf",  # Output needs to be regular lat-lon, so only works in combination with 'grid'!
    },
    "sample.nc",
)

### Jit function with parallelization

In [ ]:
from numba import jit, prange

In [ ]:
@jit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

In [ ]:
import numpy as np
input_data = np.array([1, 2, 3, 4, 5])
result = prange_test(input_data)
print(result)

### ERA5 Testing

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import geodata
from geodata.model.wind import WindInterpolationModel

In [ ]:
DS = geodata.Dataset(
    module="era5",
    weather_data_config="wind_solar_hourly",
    years=slice(2005, 2005),
    months=slice(1,1),
    bounds=[50, -3, 45, 3]
)

In [ ]:
model = WindInterpolationModel(DS)
# model.prepare()

In [ ]:
model._prepare_dataset()

In [4]:
import numpy as np
import xarray as xr
from scipy.interpolate import CubicSpline

In [8]:
filepath = "/Users/apple/.local/geodata/era5/2005/01/wind_solar_hourly.nc"

In [13]:
fp2 = "/Users/apple/Desktop/geodata/src/sample.nc"

In [9]:
ds = xr.open_dataset(filepath)

In [14]:
ds2 = xr.open_dataset(fp2)

In [15]:
ds2

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, level: 7, time: 24)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 131 132 133 134 135 136 137
  * time       (time) datetime64[ns] 2013-01-01 ... 2013-01-01T23:00:00
Data variables:
    u          (time, level, latitude, longitude) float32 ...
    v          (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-14 19:57:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [11]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 25, latitude: 21, time: 744)
Coordinates:
  * longitude  (longitude) float32 -3.0 -2.75 -2.5 -2.25 ... 2.25 2.5 2.75 3.0
  * latitude   (latitude) float32 50.0 49.75 49.5 49.25 ... 45.5 45.25 45.0
  * time       (time) datetime64[ns] 2005-01-01 ... 2005-01-31T23:00:00
Data variables:
    u100       (time, latitude, longitude) float32 ...
    v100       (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    ro         (time, latitude, longitude) float32 ...
    stl4       (time, latitude, longitude) float32 ...
    ssr        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    ssrd       (time, latitude, longitude) float32 ...
    tisr       (time, latitude, longitude) float32 ...
    fdir       (time, latitude, longitude) float32 ...
    fsr        (time, latitude, longitude) float32 ...
    z          (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-19 02:35:33 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [2]:
def interpolate_wind_speed(file_path, target_height):
    # Open the netCDF file
    ds = xr.open_dataset(file_path)
    
    # Extract the necessary variables
    u = ds['u'].values  # u-component of wind
    v = ds['v'].values  # v-component of wind
    levels = ds['level'].values  # model levels
    
    # Compute the wind speed
    wind_speed = np.sqrt(u**2 + v**2)
    
    # Interpolate the wind speed to the target height
    # Note: This is a toy example and does not take into account the actual height of each model level.
    # In a real application, you would need to convert the model levels to actual heights.
    cs = CubicSpline(levels, wind_speed, axis=1)
    interpolated_wind_speed = cs(target_height)
    
    return interpolated_wind_speed

In [16]:
interpolated_wind_speed = interpolate_wind_speed(fp2, 10)

In [17]:
interpolated_wind_speed

array([[[-2.52345811e+03, -2.52345811e+03, -2.52345811e+03, ...,
         -2.52345811e+03, -2.52345811e+03, -2.52345811e+03],
        [ 3.26551918e+03,  2.54338168e+03,  2.27378029e+03, ...,
          3.24848473e+03,  1.64116047e+03,  3.41739384e+03],
        [ 1.13119353e+04,  1.04486284e+04,  1.20172547e+04, ...,
          1.11351537e+04,  1.16217688e+04,  1.11663768e+04],
        ...,
        [ 2.50311617e+04,  2.46331274e+04,  2.38168196e+04, ...,
          2.41615221e+04,  2.44540503e+04,  2.54627565e+04],
        [ 3.26234124e+04,  3.39463727e+04,  3.32252040e+04, ...,
          3.37773340e+04,  3.34363088e+04,  3.29387114e+04],
        [ 3.75128877e+04,  3.75128877e+04,  3.75128877e+04, ...,
          3.75128877e+04,  3.75128877e+04,  3.75128877e+04]],

       [[ 8.58767181e+03,  8.58767181e+03,  8.58767181e+03, ...,
          8.58767181e+03,  8.58767181e+03,  8.58767181e+03],
        [ 8.74491120e+03,  7.64019544e+03,  7.62702534e+03, ...,
          8.85671397e+03,  8.02567445e